In [ ]:
from pydantic import BaseModel, ConfigDict, Discriminator
from typing import Annotated, Union


class OBIBaseModel(BaseModel):
    model_config = ConfigDict(extra="forbid")
    type: str

    def __init_subclass__(cls, **kwargs) -> None:
        """Dynamically set the `type` field to the class name."""
        super().__init_subclass__(**kwargs)
        cls.__annotations__["type"] = Literal[cls.__qualname__]

    def __str__(self) -> str:
        """Return a string representation of the OBIBaseModel object."""
        return self.__repr__()



class Cat(OBIBaseModel):
    name: str


class DogType(OBIBaseModel):
    breed: str


class FirstDogType(DogType):
    breed: str = "Golden Retriever"


class SecondDogType(DogType):
    breed: str = "Poodle"


class Dog(OBIBaseModel):
    name: str
    dog_type: Annotated[Union[FirstDogType, SecondDogType], Discriminator('type')]


# Discriminated Union
Animal = Annotated[Union[Cat, Dog], Discriminator('type')]
from pydantic import TypeAdapter
adapter = TypeAdapter(Animal)

# Example input
json_data = {
    "type": "Dog",
    "name": "Buddy",
    "dog_type": {
        "type": "SecondDogType"
    }
}

# Deserialize
animal = adapter.validate_python(json_data)
print("ANIMAL")
print(animal)
print(type(animal.dog_type))
print("\n")

print("DOG")
dog = Dog.model_validate(json_data)
print(dog)
print(type(dog.dog_type))
print("\n")

# Serialize
print(animal.model_dump())

ANIMAL
Dog(name='Buddy', dog_type=SecondDogType(breed='Poodle', type='SecondDogType'), type='Dog')
<class '__main__.SecondDogType'>


DOG
Dog(name='Buddy', dog_type=SecondDogType(breed='Poodle', type='SecondDogType'), type='Dog')
<class '__main__.SecondDogType'>


{'name': 'Buddy', 'dog_type': {'breed': 'Poodle', 'type': 'SecondDogType'}, 'type': 'Dog'}
